# 3 - RAG et chatbot

In [1]:
# installation des bibliothèeques
!pip install pyPDF2
!pip install faiss-cpu
!pip install sentence_transformers
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.5 MB/s eta 0:00:00


In [5]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [3]:
# jeton HuggingFace

import os
MYHFKEY = "my_key"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = MYHFKEY

In [6]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings

## 3-1 Interroger le LLM

In [8]:
# definition d'un prompt template avec un appel à un model google/flan gratuit

template = """ {question}"""
prompt_template = PromptTemplate(input_variables=["question"], template=template)
llm=HuggingFaceHub(repo_id="google/flan-t5-large",
model_kwargs={"temperature":1e-10})
chain = LLMChain(llm=llm, prompt=prompt_template)
myQuery = "Quelle est la section minimale d'écoulement d'un trop-plein?"
chain.invoke(myQuery) # la réponse est vide

{'question': "Quelle est la section minimale d'écoulement d'un trop-plein?",
 'text': '  '}

## 3-2 Import de fichier pdf

In [21]:
!ls /content/

dtu.pdf  sample_data


In [22]:
# import du dtu étanchéité 43.1 (topic : étanchéité)

newdata = PdfReader('/content/dtu.pdf')
allContent = ""
for i, page in enumerate(newdata.pages):
  content = page.extract_text()
  if content:
    allContent += content

In [26]:
# vérification du chargement

print("\nContenu de la première page :")
print(newdata.pages[86].extract_text())


Contenu de la première page :
— 87 — NF P 84-204-1-1
Si la galerie comporte un c ouvercle (ou une grille), ce dernier doit être ajouré et la section totale des ouvertures
du couvercle (ou de la grille) et de la  galerie garde-grève doit être supérieu re de 50 % à celle de l'entrée d'eau.
Dans le cas de protection dure ou asphal te, la section des ouvertur es de la grille doit êt re au moins égale à celle
de l'entrée d'eau.
Dans le cas particulier des toitures-t errasses avec isolation inversée, l’A vis Technique de l’is olant prévoit les
dispositions spécifiques c oncernant les garde-grèves.
Légende
1 Pare gravier
2 Étanchéité3 Gravillons
4 Revêtement d’étanchéité autoprotégé
5 Crapaudine
Figure 47 — Entrée d'eaux pluviales — 
Exemple de crapaudine et de galerie garde-grève
8.6.4 Trop-pleins
NOTE 1 Du fait que le niveau du trop-plein est inférieur à celu i du sommet du relevé en son point le plus bas, le fonction-
nement du trop-plein peut intervenir en  dehors des cas d'engorgement d

In [34]:
print("\nContenu de la première page :")
print(newdata.pages[88].extract_text())


Contenu de la première page :
— 89 — NF P 84-204-1-1
8.7 Étanchéité au droit des traversées et pénétrations
Les DPM doivent définir l’emplacement des traversées et le principe de leur raccordement en fonction :
— de la destination de la toiture-terrasse ;— de la distance entre émergences (voir NF P 10-203-1 [référence DTU 20.12]).
8.7.1 Raccordement des tuyaux de ventilation de chute à l'étanchéité
8.7.1.1 Cas où le tuyau débouche en toiture sans dé en béton
Le raccordement se fait par l'intermédiaire d'une pièce en plomb de 2,5 mm d'épaisseur. Cette pièce comporte
une platine et un manchon assemblés l'un à l'autre par une soudure étanche. La distance entre le bord de la pla-tine et le manchon est au minimum de 0,12 m. La haut eur du manchon est telle que son niveau supérieur dépasse
d'une hauteur minimale de 0,15 m le niveau supérieur de la protection. Le manchon est adossé au tuyau métalli-que ou au fourreau méta llique solidaire du gros œuvre (voir figure 50 et  51). La platine est

In [27]:
# le document étant de taille conséquente, on le découpe en chuncks

text_splitter = CharacterTextSplitter(separator="\n",
chunk_size=200,
chunk_overlap= 20,
length_function= len)
finalAllContent = text_splitter.split_text(allContent)
finalAllContent

['© AFNOR 2004 AFNOR 2004 1er tirage 2004-10-F© AFNOR 2004 — Tous droits réservésFA136499 ISSN 0335-3931\nNF P 84-204-1-1\nNovembre 2004\nIndice de classement: P 84-204-1-1',
 'Éditée et diffusée par l’Association Française de Normalisation (AFNOR) — 11, avenue Francis de Pressensé — 93571 Saint-Denis La P laine Cedex',
 'Tél. : + 33 (0)1 41 62 80 00 — Fax : + 33 (0)1 49 17 90 00 — www.afnor.fr\nDiffusée par le CSTB 4, av. du Recteur-Poincaré 75782 Paris Cedex 16',
 'Tél. : 01 40 50 28 28 — Tél. international : + 33 1 40 50 28 28ICS : 91.060.20 ; 91.120.30\nDTU 43.1 — Travaux de bâtiment\nÉtanchéité des toitures-terrasses',
 'et toitures inclinées avec éléments porteurs \nen maçonnerie en climat de plaine\nPartie 1-1 : Cahier  des clauses techniques',
 'E : DTU 43.1 — Building works — Impermeabilit y of flat roofs and inclined roofs \nwith a masonry structural member in a low- land climate — Part 1-1: Contract bill \nof technical clauses',
 'of technical clauses\nD : DTU 43.1 — Bauarbe

## 3-3 Embedding


In [28]:
# vectorisation des chunks avec un embedding de Hugging face

embeddings = HuggingFaceEmbeddings()

<ipython-input-28-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-28-d0c9174021d8>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 3-4 Vectorisation

In [29]:
# on importe les donnees vectorisees du pdf

docSearch = FAISS.from_texts(finalAllContent, embeddings) # on utilise la librairie Open Source FAISS
newChain = load_qa_chain(llm, chain_type="stuff")
docs = docSearch.similarity_search(myQuery)

<ipython-input-29-a390b1ab95e8>:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  newChain = load_qa_chain(llm, chain_type="stuff")


## 3-5 Ré-interrogation de LLM

In [30]:
newChain.run(input_documents=docs, question=myQuery)

'28 cm 2'

In [31]:
newChain.run(input_documents=docs, question="Quelle est la section minimale d'écoulement d'un trop-plein?") #page 86 - ok

'28 cm 2'

In [33]:
newChain.run(input_documents=docs, question="Comment fonctionne un trop-plein?") #mixte anglais et francais ? pourquoi ne cite t'il pas le dtu ? # page 86

'It is composed of a plate and a moignon assemblés between them by'

In [35]:
newChain.run(input_documents=docs, question="Quelle est la hauteur des tuyaux de ventilation de chute?") # page 88 - faux,  la hauteur du manchon est telle que son niveau supérieur dépasse d'une hauteur minimale de 0,15 m le niveau supérieur de la protection


'0,05 m'

In [36]:
newChain.run(input_documents=docs, question="Quelle est la distance entre le bord de la platine de ventilation de chute et le manchon ?") # page 88 - faux, c'est 0.12cm



'0,05 m'

## 3-6 Chargement d' autre DTU